In [2]:
import sys
sys.path.append('/home/aw1223/ip/agile/sdk')
%load_ext autoreload
%autoreload 2
from sdk.ample import Ample

from sdk.graphs.random_graph import RandomGraph
#TODO change to just models in file sturcture
from sdk.models.models import GCN_Model, GAT_Model, GraphSAGE_Model, GIN_Model, GCN_MLP_Model, MLP_Model, Edge_Embedding_Model, Interaction_Net_Model
from sdk.graph_tracer import GraphTracer




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#Import neural lam
ample = Ample()
edge = True
#TODO fix this : If model does not use edges, dont set edges to be true - will brrak things 
graph = RandomGraph(num_nodes=10, avg_degree=1, num_channels=32, graph_precision="FLOAT_32",edge_dim=32,edges = edge) #TODO add var


In [ ]:
import argparse

# Manually create the args Namespace object with the desired parameters
args = argparse.Namespace(
    dataset='meps_example',
    model='graph_lam',
    subset_ds=0,
    seed=42,
    n_workers=4,
    epochs=200,
    batch_size=4,
    load=None,
    restore_opt=0,
    precision=32,
    graph='1level',
    hidden_dim=64,
    hidden_layers=1,
    processor_layers=4,
    mesh_aggr='sum',
    output_std=0,
    ar_steps=1,
    control_only=0,
    loss='wmse',
    step_length=3,
    lr=0.001,
    val_interval=1,
    eval='test',
    n_example_pred=1
)

print('args', args)
print('args', args.dataset)

args Namespace(dataset='meps_example', model='graph_lam', subset_ds=0, seed=42, n_workers=4, epochs=200, batch_size=4, load=None, restore_opt=0, precision=32, graph='1level', hidden_dim=64, hidden_layers=1, processor_layers=4, mesh_aggr='sum', output_std=0, ar_steps=1, control_only=0, loss='wmse', step_length=3, lr=0.001, val_interval=1, eval='test', n_example_pred=1)
args meps_example


In [ ]:
#Trace model, find the constituent models e.g GCN, Interaction net and create a data flow graph

sys.path.append('/home/aw1223/ip/agile/imports/neural-lam')


from neural_lam.models.graph_lam import GraphLAM

model_cls = GraphLAM(args)

tracer = GraphTracer(model)

# tracer.get_input_output_layers()
# # # tracer.print_input_output_layers()
# # tracer.draw()
# ample.initialize_memory(model,graph)
# ample.sim()


KeyboardInterrupt



In [ ]:
# ample.initialize_memory(model,graph)
# ample.sim()

1it [00:00, 12192.74it/s]
2it [00:00, 16743.73it/s]


cd /home/aw1223/ip/agile/hw/sim
==== Running command: cd /home/aw1223/ip/agile/hw/sim; make run_sim
make -f Makefile.modelsim run_sim
make[1]: Entering directory '/home/aw1223/ip/agile/hw/sim'
set -o pipefail; LIBPYTHON_LOC=/home/aw1223/anaconda3/envs/ample/lib/libpython3.11.so.1.0 MODULE=runner TESTCASE= TOPLEVEL="work.top_wrapper_tb" \
GPI_EXTRA= TOPLEVEL_LANG=verilog \
 /mnt/applications/mentor/modelsim-2019.2/modelsim/modeltech/linux_x86_64/vsim -c -64 -l sim_build/modelsim.log -onfinish exit -do sim_build/../simulate_no_gui.do -quiet -suppress vsim-3015  -pli /home/aw1223/anaconda3/envs/ample/lib/python3.11/site-packages/cocotb/libs/libcocotbvpi_modelsim.so \
 \
work.work_opt \
 2>&1 | tee sim_build/simulate.log
Reading pref.tcl

# 2019.2

# vsim -c -l sim_build/modelsim.log -onfinish exit -do "sim_build/../simulate_no_gui.do" -quiet -suppress vsim-3015 -pli "/home/aw1223/anaconda3/envs/ample/lib/python3.11/site-packages/cocotb/libs/libcocotbvpi_modelsim.so" work.work_opt 
# Start

In [ ]:
#Use initalize to construct memory map for each model and then string together the dataflow between models

In [ ]:
#Program Agile using device